First all needed modules are imported

In [ ]:
import numpy as np
import networkx as nx
from collections import defaultdict
import random
import pickle
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import collections
import itertools
import scipy as sp
import scipy.stats

import os

import osmnx as ox

from toysimulations import ZeroDetourBus, Stop, Request, Network
import route

Functions creating the requests according to a Possion process and passed parameters are defined

In [2]:
def req_generator_uniform(graph, num_reqs, req_rate):
    """
    Generates requests with rate=req_rate whose origin and
    destination are drawn uniformly randomly. The requests
    are generated in time as a Poisson process.
    """
    t = 0
    req_idx = 0

    while req_idx < num_reqs:
        orig, dest = random.sample(graph.nodes(), k=2)
        delta_t = np.random.exponential(1/req_rate)

        t += delta_t
        req_idx += 1
        yield Request(req_idx, t, orig, dest)
        

def req_generator_jump(graph,num_reqs,req_rate):
    
    """
    Alteration of req_generator_uniform realising a jump in request rate
    different options: in this case the second req_rate is two times the first one and the time of the jump is random,
    specific jumptime and req_rate also possible, would mean we have to adapt the arguments as well
    """
    t=0
    req_idx=0
    
    num_jump=0.4*num_reqs     #random.random()*num_reqs
    req_jump=2*req_rate #could be other number or random as well
    
    while req_idx < num_reqs:
        
        if req_idx<num_jump:
            orig, dest = random.sample(graph.nodes(), k=2)
            delta_t = np.random.exponential(1/req_rate)

            t += delta_t
            req_idx += 1
            yield Request(req_idx, t, orig, dest)
            
        if req_idx>=num_jump:
            orig, dest = random.sample(graph.nodes(), k=2)
            delta_t = np.random.exponential(1/req_jump)

            t += delta_t
            req_idx += 1
            yield Request(req_idx, t, orig, dest)

def req_generator_jumping2(graph, num_reqs, req_rate):
    """
    Alteration from req_generator_uniform
    Generates requests with jumps in the request rate every 500 requests.
    """
    t = 0
    req_idx = 0

    while req_idx < num_reqs:
        if  req_idx % 500 == 0:
            req_rate=req_rate+(2)
        orig, dest = random.sample(graph.nodes(), k=2)
        delta_t = np.random.exponential(1/req_rate)

        t += delta_t
        req_idx += 1
        yield Request(req_idx, t, orig, dest)  

Functions starting the simulation with different request generators are defined. They return the request data dictionary and the insertion data array containing all the needed parameters for the analysis, like the stop list length of a request, the time of pick up and drop off etc.

In [ ]:
def simulate_single_request_rate(G, nG, x,network_type, l_avg):
    """
    Simulates only as single request rate x. See the docstring of
    `simulate_different_request_rates` for details on the arguments.
    """
    num_reqs = 10000
    req_rate = x/2/l_avg

    sim = ZeroDetourBus(nG,
                        req_generator_uniform(G, num_reqs, req_rate),
                        network_type, 
                        random.sample(G.nodes(), k=1)[0],
                       )
    print(f"simulating x={x}")
    sim.simulate_all_requests()
    return sim.req_data, sim.insertion_data


def simulate_two_request_rates(G, nG, x,network_type, l_avg):
    """
    Alteration from simulate_single_request_rate
    Simulates two request rates by calling req_generator_jump
    """
    num_reqs = 10000
    req_rate = x/2/l_avg

    sim = ZeroDetourBus(nG,
                        req_generator_jump(G, num_reqs, req_rate),
                        network_type, 
                        random.sample(G.nodes(), k=1)[0]
                       )
    print(f"simulating x={x}")
    sim.simulate_all_requests()
    return sim.req_data, sim.insertion_data

def simulate_many_request_rates2(G, nG, x,network_type, l_avg):
    """
    Alteration from simulate_single_request_rate.
    Simulates request rates over a wide range by calling req_generator_jumping2
    """
    num_reqs = 20000
    req_rate = x/2/l_avg

    sim = ZeroDetourBus(nG,
                        req_generator_jumping2(G, num_reqs, req_rate),
                        network_type, 
                        random.sample(G.nodes(), k=1)[0]
                       )
    print(f"simulating x={x}")
    sim.simulate_all_requests()
    return sim.req_data, sim.insertion_data

Functions are defined that draw the performance parameters from the data created by the simulation. The data is obtained from 30 simulations and then averaged.

In [3]:
def calculate_avg_jump(G, nG, x, network_type, l_avg):
    
    """
    Simulations with a jump in request rate are started.
    Performance parameters are saved for every request.
    The data from 30 simulations is averaged.
    Arrays containing the average performance parameters for every single request are returned.
    """
    
    wait_gen=[]
    stoplist_gen=[]
    times_gen=[]
    service_gen=[]
    for i in range(0,30):
        req_data_jump, insertion_data_jump=simulate_two_request_rates(G, nG, x, network_type, l_avg)
        
        wait_jump=[]
        for key in req_data_jump.keys():
            wait_jump.append(float(req_data_jump[key]['pickup_epoch'])-float(req_data_jump[key]['req_epoch']))
        wait_gen.append(wait_jump)
        
        service_jump=[]
        for key in req_data_jump.keys():
            service_jump.append(float(req_data_jump[key]['dropoff_epoch'])-float(req_data_jump[key]['req_epoch']))
        service_gen.append(service_jump)

        stoplist_jump=[]
        for element in insertion_data_jump:
            stoplist_jump.append(element[1])
        stoplist_gen.append(stoplist_jump)  
        
        times_request=[]
        for element in insertion_data_jump:
            times_request.append(element[0])
        times_gen.append(times_request)
        
    temp_wait=np.array(wait_gen)
    wait_sum=np.transpose(temp_wait)
    wait_avg=[]
    for i in wait_sum:
        wait_avg.append(np.mean(i))
        
    temp_service=np.array(service_gen)
    service_sum=np.transpose(temp_service)
    service_avg=[]
    for i in service_sum:
        service_avg.append(np.mean(i))

    temp_stoplist=np.array(stoplist_gen)
    stoplist_sum=np.transpose(temp_stoplist)
    stoplist_avg=[]
    for element in stoplist_sum:
        stoplist_avg.append(np.mean(element))
        
    temp_times=np.array(times_gen)
    times_sum=np.transpose(temp_times)
    times_avg=[]
    for i in times_sum:
        times_avg.append(np.mean(i))
        
    return wait_avg,stoplist_avg,times_avg,service_avg

def calculate_avg_jumping2(G, nG, x, network_type, l_avg):
    
    """
    Simulations a wide range of request rates jumping every 500 requests are started.
    Performance parameters are saved for every request.
    The data from 30 simulations is averaged.
    Arrays containing the average performance parameters for every single request are returned.
    """
    
    wait_gen=[]
    stoplist_gen=[]
    times_gen=[]
    service_gen=[]
    for i in range(0,30):
        req_data, insertion_data=simulate_many_request_rates2(G, nG, x, network_type=network_type, l_avg=l_avg)
        
        wait=[]
        for key in req_data.keys():
            wait.append(float(req_data[key]['pickup_epoch'])-float(req_data[key]['req_epoch']))
        wait_gen.append(wait)
        
        service=[]
        for key in req_data.keys():
            service.append(float(req_data[key]['dropoff_epoch'])-float(req_data[key]['req_epoch']))
        service_gen.append(service)

        stoplist=[]
        for element in insertion_data:
            stoplist.append(element[1])
        stoplist_gen.append(stoplist)  
        
        times_request=[]
        for element in insertion_data:
            times_request.append(element[0])
        times_gen.append(times_request)
        
    temp_wait=np.array(wait_gen)
    wait_sum=np.transpose(temp_wait)
    wait_avg=[]
    for i in wait_sum:
        wait_avg.append(np.mean(i))
        
    temp_service=np.array(service_gen)
    service_sum=np.transpose(temp_service)
    service_avg=[]
    for i in service_sum:
        service_avg.append(np.mean(i))

    temp_stoplist=np.array(stoplist_gen)
    stoplist_sum=np.transpose(temp_stoplist)
    stoplist_avg=[]
    for element in stoplist_sum:
        stoplist_avg.append(np.mean(element))
        
    temp_times=np.array(times_gen)
    times_sum=np.transpose(temp_times)
    times_avg=[]
    for i in times_sum:
        times_avg.append(np.mean(i))
        
    return wait_avg,stoplist_avg,times_avg,service_avg


The jump in request rate is applied with a starting request rate of 10 on the street network of Ingolstadt as an example.

In [4]:
#importing the graph
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)

#starting the simulation and retrieving the performance parameter data
x=10
nG = Network(G, network_type='novolcomp')
l_avg = nx.average_shortest_path_length(G)
wait_avg,stoplist_avg,times_avg,servicetime_avg=calculate_avg_jump(G, nG, x, network_type='novolcomp', l_avg=l_avg)

#defining a function to create a moving_average (will be used for the big variations in wait time)
def create_moving_avg(data):
    df=pd.DataFrame({'data':data})
    df['moving_avg']=df['data'].rolling(window =250,center=True).mean()
    print(df.head(5))
    #df['moving_avg'].plot()
    moving_avg=list(df['moving_avg'])
    return moving_avg

#creating the image of the jump in wait time
moving_wait=create_moving_avg(wait_avg)
plt.plot(moving_wait)
plt.xlabel('Request Nr.')
plt.ylabel('Waiting Time')
plt.show()

#creating the image of the jump in stop list length
plt.plot(stoplist_avg)
plt.xlabel('Request Nr.')
plt.ylabel('Stoplist Length')
plt.show()

ModuleNotFoundError: No module named 'shapely'

The data for jumping request rates is created and viziualized as an example for the street network of Ingolstadt

In [5]:
#importing the garph
print("doing Ingolstadt")
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)

#starting the simulations and retrieving the performance parameter data    
nG = Network(G,network_type='novolcomp')
x=1
l_avg = nx.average_shortest_path_length(G)
wait_avg,stoplist_avg,times_avg,servicetime_avg=calculate_avg_jumping2(G, nG, x, network_type='novolcomp', l_avg=l_avg)

#creating an array with the request rate always going up by two every 500 steps
x_rate=[None]*len(wait_avg)
rate=1
for i in range(0,len(wait_avg)):
    x_rate[i]=rate
    if i%500==0:
        rate=rate+2

#creating the plots of the jumping request rates
plt.plot(x_rate,stoplist_avg)
plt.xlabel('Request Rate')
plt.ylabel('Stop List Length')
plt.show()

moving_wait=create_moving_avg(wait_avg)
plt.plot(x_rate,moving_wait)
plt.xlabel('Request Rate')
plt.ylabel('Waiting Time')

doing Ingolstadt


ModuleNotFoundError: No module named 'shapely'

Subsequently, the curve is fitted to a function to obtain the value of a possible asymptote.


In [ ]:
from scipy.optimize import curve_fit

#define function fo do the fitting

def fit_function(x,a,b):
    y=a/x+b
    return y

#fit data to function and retrieve parameters
parameters, covariance = curve_fit(fit_function, x_rate, wait_avg)
print(parameters)
fit_A=parameters[0] #parameter a
fit_B=parameters[1] # parameter b, asymptote for which we were looking

#create the graph showing the fit function
fit_y = fit_function(x_rate, fit_A, fit_B)
plt.plot(x_rate, create_moving_avg(wait_times), color='Blue', label='data')
plt.plot(x_rate, fit_y, color='Red', label='fit')
plt.xlabel('Request Rate')
plt.ylabel('Wait Time')
plt.legend()

